In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from dotenv import load_dotenv
import os

load_dotenv()

# Your API key
api_key = os.getenv('API_KEY')
season = "2024"  # Specify the season

# Base URLs for the endpoints
game_base_url = "https://api.sportsdata.io/v3/nba/scores/json/ScoresBasicFinal/"
season_stats_url = f"https://api.sportsdata.io/v3/nba/scores/json/TeamSeasonStats/{season}?key={api_key}"

# Initialize an empty list to store game data
all_games = []

# Function to fetch data for a specific date
def fetch_data_for_date(date):
    formatted_date = date.strftime("%Y-%m-%d")
    url = f"{game_base_url}{formatted_date}?key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for {formatted_date}")
        return []

# Fetch data for the last 30 playable days
days = 30
date = datetime.now()

while len(all_games) < days:
    games = fetch_data_for_date(date)
    if games:  # Only add the date if there were games played
        all_games.extend(games)
    date -= timedelta(days=1)

# Convert the data into a DataFrame
df = pd.DataFrame(all_games)

# Fetch the season stats data
response = requests.get(season_stats_url)
season_stats = response.json()

# Print the entire object to see its structure
print("Season Stats Response:", season_stats)

# Check if the response is a list and has content
if isinstance(season_stats, list) and len(season_stats) > 0:
    # Convert to DataFrame
    season_stats_df = pd.DataFrame(season_stats)

    # Select relevant columns from the season stats for merging
    stats_columns = [
    "StatID", "TeamID", "SeasonType", "Season", "Name", "Team", "Wins", "Losses",
    "Possessions", "Games", "FantasyPoints", "Minutes", "FieldGoalsMade",
    "FieldGoalsAttempted", "FieldGoalsPercentage", "TwoPointersMade",
    "TwoPointersAttempted", "TwoPointersPercentage", "ThreePointersMade",
    "ThreePointersAttempted", "ThreePointersPercentage", "FreeThrowsMade",
    "FreeThrowsAttempted", "FreeThrowsPercentage", "OffensiveRebounds",
    "DefensiveRebounds", "Rebounds", "Assists", "Steals", "BlockedShots",
    "Turnovers", "PersonalFouls", "Points"
    ]
    # Check which columns actually exist in the DataFrame
    existing_columns = [col for col in stats_columns if col in season_stats_df.columns]

    # Select only the existing columns
    season_stats_df = season_stats_df[existing_columns]


    # Merge season stats for the home team
    df = df.merge(season_stats_df, how='left', left_on='HomeTeamID', right_on='TeamID', suffixes=('', '_Home'))

    # Merge season stats for the away team
    df = df.merge(season_stats_df, how='left', left_on='AwayTeamID', right_on='TeamID', suffixes=('', '_Away'))

    # Drop any redundant columns after merging
    df.drop(columns=['TeamID', 'TeamID_Away'], inplace=True)

    # Save the data to a CSV file (Optional: remove this line if you don't want to save)
    csv_filename = "nba_last_30_days.csv"
    df.to_csv(csv_filename, index=False)

    print(f"Data saved to {csv_filename}")
else:
    print("The season stats data was not returned in the expected format.")


Season Stats Response: [{'StatID': 1115404, 'TeamID': 3, 'SeasonType': 1, 'Season': 2024, 'Name': 'Atlanta Hawks', 'Team': 'ATL', 'Wins': 36, 'Losses': 46, 'OpponentPosition': None, 'Possessions': 2777.5, 'GlobalTeamID': 20000003, 'Updated': '2024-06-25T22:42:38', 'Games': 82, 'FantasyPoints': 6182.2, 'Minutes': 6707, 'Seconds': 19, 'FieldGoalsMade': 1192.8, 'FieldGoalsAttempted': 2563.4, 'FieldGoalsPercentage': 15.7, 'EffectiveFieldGoalsPercentage': 18.3, 'TwoPointersMade': 812.6, 'TwoPointersAttempted': 1518.3, 'TwoPointersPercentage': 18.1, 'ThreePointersMade': 380.2, 'ThreePointersAttempted': 1045.1, 'ThreePointersPercentage': 12.3, 'FreeThrowsMade': 513.8, 'FreeThrowsAttempted': 644.2, 'FreeThrowsPercentage': 26.9, 'OffensiveRebounds': 346.1, 'DefensiveRebounds': 892.0, 'Rebounds': 1238.1, 'OffensiveReboundsPercentage': None, 'DefensiveReboundsPercentage': None, 'TotalReboundsPercentage': None, 'Assists': 736.8, 'Steals': 207.9, 'BlockedShots': 124.7, 'Turnovers': 353.5, 'Personal

In [ ]:
# downlod file
from google.colab import files
files.download("nba_last_30_days.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Library imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv('nba_last_30_days.csv')


In [ ]:
# Proprocess data

#  Fill missing values with the mean of each column (only in numeric columns)
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())


# Example of columns: 'HomeTeam', 'AwayTeam', 'HomeTeamScore', 'AwayTeamScore'
# Create a target variable 'WinningTeam' where 1 means home team won, 0 means away team won
df['WinningTeam'] = np.where(df['HomeTeamScore'] > df['AwayTeamScore'], 1, 0)

# Select relevant features for the model (you can add more if available)
features = ['HomeTeamScore', 'AwayTeamScore']

X = df[features]
y = df['WinningTeam']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Train logistic model

# Initialize the logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print a detailed classification report
print(classification_report(y_test, y_pred))


Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         4

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7



In [ ]:
!pip install praw

In [ ]:
# SOCIAL MEDIA OUTLOOK
import praw

REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET')
REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT')

# Initialize the Reddit instance
reddit = praw.Reddit(client_id=REDDIT_CLIENT_ID,
                     client_secret=REDDIT_CLIENT_SECRET,
                     user_agent=REDDIT_USER_AGENT)

# Dictionary mapping team names to their subreddit names
team_subreddits = {
    "Atlanta Hawks": "AtlantaHawks",
    "Boston Celtics": "bostonceltics",
    "Brooklyn Nets": "GoNets",
    "Charlotte Hornets": "CharlotteHornets",
    "Chicago Bulls": "chicagobulls",
    "Cleveland Cavaliers": "clevelandcavs",
    "Dallas Mavericks": "Mavericks",
    "Denver Nuggets": "denvernuggets",
    "Detroit Pistons": "DetroitPistons",
    "Golden State Warriors": "warriors",
    "Houston Rockets": "rockets",
    "Indiana Pacers": "pacers",
    "Los Angeles Clippers": "LAClippers",
    "Los Angeles Lakers": "lakers",
    "Memphis Grizzlies": "memphisgrizzlies",
    "Miami Heat": "heat",
    "Milwaukee Bucks": "MkeBucks",
    "Minnesota Timberwolves": "timberwolves",
    "New Orleans Pelicans": "NOLAPelicans",
    "New York Knicks": "NYKnicks",
    "Oklahoma City Thunder": "Thunder",
    "Orlando Magic": "OrlandoMagic",
    "Philadelphia 76ers": "sixers",
    "Phoenix Suns": "suns",
    "Portland Trail Blazers": "ripcity",
    "Sacramento Kings": "kings",
    "San Antonio Spurs": "NBASpurs",
    "Toronto Raptors": "torontoraptors",
    "Utah Jazz": "UtahJazz",
    "Washington Wizards": "washingtonwizards",
}

# Function to get posts from a team's subreddit
def get_team_posts(team_name):
    subreddit_name = team_subreddits.get(team_name)
    posts = []

    if subreddit_name:
        subreddit = reddit.subreddit(subreddit_name)
        fetched_posts = subreddit.hot(limit=10)  # Get top 10 hot posts
        print(f"Top 10 posts from r/{subreddit_name}:\n")
        for post in fetched_posts:
            post_content = f"Title: {post.title}\nScore: {post.score}\nURL: {post.url}\n"
            posts.append(post_content)
            print(post_content)
    else:
        print(f"Subreddit for {team_name} not found.")

    return posts

# Example usage
team_name = "Los Angeles Lakers"  # You can change this to any team
posts = get_team_posts(team_name)

# The 'posts' list now contains the top 10 posts from the subreddit's hot section


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Top 10 posts from r/lakers:

Title: Daily Lakers Offseason Discussion Thread
Score: 3
URL: https://www.reddit.com/r/lakers/comments/1eotq0d/daily_lakers_offseason_discussion_thread/

Title: [LakersNation] Lakers News: Anthony Davis Believes Olympics Will Help Prepare Him For 2024-25 Season
Score: 77
URL: https://lakersnation.com/lakers-news-anthony-davis-believes-olympics-will-help-prepare-him-for-2024-25-season/

Title: The script has been written...
Score: 156
URL: https://i.redd.it/v33metlz7uhd1.png

Title: Go for Gold on 8/10/24!! 
Score: 59
URL: https://i.redd.it/irbn52bm8vhd1.jpeg

Title: Bring home the gold  8/10/24
Score: 264
URL: https://i.redd.it/4kp6fuzfrrhd1.jpeg

Title: Jovan Buha gives big update on the D'Angelo Russell trade market. While most teams are not interested, the Raptors and Nets are reportedly open to trade
Score: 11
URL: https://sportsnaut.com/los-angeles-lakers-dangelo-russell-low-trade-value/

Title: KD remains undefeated on Twitter, putting Nuggets fans in

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
# SENTIMENT ANALYSIS
import dotenv
import os
from openai import OpenAI

load_dotenv()

# OpenAI API Key and client
api_key = os.getenv(OPENAI_API_KEY)

client = OpenAI(
    api_key=api_key
)

# Function to summarize fan sentiment
def summarize_fan_sentiment(posts):
    # Concatenate the posts into a single string for analysis
    posts_content = "\n\n".join(posts)

    # Create the chat completion
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are an AI trained to analyze sentiment from social media posts."
            },
            {
                "role": "user",
                "content": f"Analyze the following Reddit posts for their sentiment towards the team:\n{posts_content}"
            }
        ],
        model="gpt-4o-mini",
    )

    # Extract the summary from the response
    summary = chat_completion.choices[0].message.content

    return summary

sentiment_summary = summarize_fan_sentiment(posts)
print(sentiment_summary)

Here’s a sentiment analysis of the provided Reddit posts regarding the Lakers:

1. **Title: Daily Lakers Offseason Discussion Thread**
   - **Score**: 3
   - **Sentiment**: Neutral/Low Positive. The low score suggests a lack of engagement or enthusiasm about offseason discussions.

2. **Title: [LakersNation] Lakers News: Anthony Davis Believes Olympics Will Help Prepare Him For 2024-25 Season**
   - **Score**: 77
   - **Sentiment**: Positive. The topic of Anthony Davis preparing for the upcoming season is likely to generate optimism among fans, leading to higher positive sentiment.

3. **Title: The script has been written...**
   - **Score**: 156
   - **Sentiment**: Positive. A score of 156 indicates enthusiasm or excitement, possibly alluding to a strong narrative or expectation surrounding the team.

4. **Title: Go for Gold on 8/10/24!!**
   - **Score**: 59
   - **Sentiment**: Positive. The exclamation marks suggest excitement and eagerness regarding the upcoming Olympics, indicative

In [ ]:
!pip install google-api-python-client pytubefix

In [ ]:
from googleapiclient.discovery import build
from pytubefix import YouTube
from pytubefix.cli import on_progress


# Function to search YouTube for a specific channel
def search_youtube_channel(query, channel_id, max_results=10):
    API_KEY = os.getenv(YOUTUBE_API_KEY)
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    # Search query specifically targeting basketball-related content
    request = youtube.search().list(
        part="snippet",
        maxResults=max_results,
        q=query + ' "at" ("FULL GAME HIGHLIGHTS" OR "FULL GAME 1 HIGHLIGHTS" OR "FULL GAME 2 HIGHLIGHTS" OR "FULL GAME 3 HIGHLIGHTS" OR "FULL GAME 4 HIGHLIGHTS" OR "FULL GAME 5 HIGHLIGHTS" OR "FULL GAME 6 HIGHLIGHTS" OR "FULL GAME 7 HIGHLIGHTS") -2K -gaming -olympic -showcase -politics -news -interview -commentary',
        type="video",
        channelId=channel_id  # Specific channel ID
    )

    response = request.execute()

    videos = []
    for item in response.get('items', []):
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        videos.append((title, video_url))
        print(f"{title}\n{video_url}\n")

    return videos

# List of channels to search
channels = {
    "NBA": "UCWJ2lWNubArHWmf3FIHbfcQ",  # Official NBA channel
}

# Combine results from multiple channels
all_videos = []
for channel_name, channel_id in channels.items():
    print(f"Searching in {channel_name} channel...")
    videos = search_youtube_channel("NBA game highlights", channel_id)
    all_videos.extend(videos)

# Download the first video using pytubefix
if all_videos:
    first_video_url = all_videos[0][1]
    try:
        yt = YouTube(first_video_url, on_progress_callback=on_progress)

        # Select the stream with the highest resolution
        ys = yt.streams.get_highest_resolution()
        ys.download(filename='nba_highlight.mp4')

        print("First video downloaded as nba_highlight.mp4")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


Searching in NBA channel...
CELTICS at WARRIORS | FULL GAME 5 NBA FINALS HIGHLIGHTS | June 13, 2022
https://www.youtube.com/watch?v=fTpZSpG0_vg

CELTICS at WARRIORS | FULL GAME 2 NBA FINALS HIGHLIGHTS | June 5, 2022
https://www.youtube.com/watch?v=GhkI0wYrAZM

WARRIORS at CELTICS | FULL GAME 4 NBA FINALS HIGHLIGHTS | June 10, 2022
https://www.youtube.com/watch?v=B-5StlRZInc

WARRIORS at CELTICS | FULL GAME 6 NBA FINALS HIGHLIGHTS | June 16, 2022
https://www.youtube.com/watch?v=AOYACk7m7Fk

WARRIORS at CELTICS | FULL GAME 3 NBA FINALS HIGHLIGHTS | June 8, 2022
https://www.youtube.com/watch?v=lFovcuvoGCg

CELTICS at WARRIORS | FULL GAME 1 NBA FINALS HIGHLIGHTS | June 2, 2022
https://www.youtube.com/watch?v=FORXBT6GFa0

BUCKS at SUNS | FULL GAME 5 NBA FINALS HIGHLIGHTS | July 17, 2021
https://www.youtube.com/watch?v=NQVD6ddpwco

SUNS at BUCKS | FULL GAME 4 NBA FINALS HIGHLIGHTS | July 14, 2021
https://www.youtube.com/watch?v=a_39qOCpDco

Stephen Curry NBA RECORD 16 THREES &amp; 50 PTS at 

In [ ]:
!pip install moviepy opencv-python pillow

In [ ]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from PIL import Image
import base64
from openai import OpenAI

import dotenv
import os

load_dotenv()

# OpenAI API Key and client
api_key = os.getenv(OPENAI_API_KEY)

client = OpenAI(
    api_key=api_key
)

# Step 1: Extract the first 10 seconds of the video
def extract_video_and_audio(video_path, output_video_path, output_audio_path):
    clip = VideoFileClip(video_path).subclip(0, 10)
    clip.write_videofile(output_video_path, codec="libx264")
    clip.audio.write_audiofile(output_audio_path)

# Step 2: Extract 9 frames and create a 3x3 grid
def create_photo_grid(video_path, output_image_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_extract = np.linspace(0, frame_count-1, 9, dtype=int)

    images = []
    for i in frames_to_extract:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            images.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    cap.release()

    # Create a 3x3 grid
    grid_image = Image.new('RGB', (images[0].shape[1]*3, images[0].shape[0]*3))
    for i, img in enumerate(images):
        img_pil = Image.fromarray(img)
        grid_image.paste(img_pil, ((i % 3) * img_pil.width, (i // 3) * img_pil.height))

    grid_image.save(output_image_path)

# Step 3: Transcribe the first 10 seconds of audio using Whisper
def transcribe_audio(audio_path):
    with open(audio_path, "rb") as audio_file:
        # Create the transcription
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text"  # Ensuring we're requesting a text format response
        )

        # Debugging: Print the type and content of the transcription variable
        print(f"Transcription type: {type(transcription)}")
        print(f"Transcription content: {transcription}")

        # If transcription is a dictionary (as expected), return the text
        if isinstance(transcription, dict) and 'text' in transcription:
            return transcription['text']
        # If transcription is a string, return it directly
        elif isinstance(transcription, str):
            return transcription
        else:
            raise ValueError("Unexpected response format from the API")


# Step 4: Use GPT-4 with Vision to analyze the photo grid and transcription
def analyze_image_and_text(image_path, transcription):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    image_b64 = base64.b64encode(image_data).decode('utf-8')

    # Creating a detailed and targeted prompt
    prompt = (
        "You are provided with an image from an NBA playoff game and the corresponding audio transcription. "
        "Analyze the image and the transcription to determine key moments, identify players, actions, and any significant events in the game. "
        "Also, analyze the setting of the game and provide a detailed summary of what is happening."
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}", "detail": "high"}},
                    {"type": "text", "text": transcription}
                ]
            }
        ],
        max_tokens=500,  # Increase max tokens if you expect a detailed response
    )

    return response.choices[0].message.content

# Example usage
video_path = "nba_highlight.mp4"  # Path to the downloaded video
output_video_path = "short_video.mp4"
output_audio_path = "short_audio.mp3"
output_image_path = "photo_grid.jpg"

extract_video_and_audio(video_path, output_video_path, output_audio_path)
create_photo_grid(output_video_path, output_image_path)
transcription = transcribe_audio(output_audio_path)

# Analyze the image and transcription
analysis = analyze_image_and_text(output_image_path, transcription)
print(analysis)


Moviepy - Building video short_video.mp4.
MoviePy - Writing audio in short_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video short_video.mp4



Moviepy - Done !
Moviepy - video ready short_video.mp4
MoviePy - Writing audio in short_audio.mp3


MoviePy - Done.
Transcription type: <class 'str'>
Transcription content: And I would block out all the positive suggestions by so many. Listen to your coaches, your teammates, but most importantly, trust yourself.

In the provided image, we see an NBA playoff game likely from the Finals, given the "NBA Finals" logo on the court. The setting is Chase Center, home of the Golden State Warriors. The atmosphere appears electric, with a packed crowd and a sea of yellow and green jerseys, indicating the teams involved are the Golden State Warriors and the Boston Celtics.

### Key Moments & Actions
1. **Tip-Off**: The game is in its early moments, with players positioned for the tip-off. This is a critical moment as it sets the tone for the start of the game.
  
2. **Teams**: The players in the image are wearing their team colors. The Warriors are in white (with some yellow accents), while the Celtics are in green, which helps identify the teams competing.

3. **Crowd Engagement**: The visible

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming you've already done the data preprocessing and training as in your provided script

# Step 1: Get season data prediction (logistic regression)
def predict_season_outcome(home_team_score, away_team_score, model):
    game_data = np.array([[home_team_score, away_team_score]])
    # Predict probability of home team winning
    season_prob = model.predict_proba(game_data)[0][1]  # [0][1] is the probability for class 1 (home win)
    return season_prob

# Step 2: Integrate sentiment and video analysis scores (already calculated)
def get_final_prediction(home_team_score, away_team_score, model, video_score_home, video_score_away, sentiment_score_home, sentiment_score_away):
    # Get the season prediction
    season_prob_home = predict_season_outcome(home_team_score, away_team_score, model)
    season_prob_away = 1 - season_prob_home

    # Weights for the different factors
    weights = {
        'season': 0.5,
        'video': 0.3,
        'sentiment': 0.2
    }

    # Normalize the video and sentiment scores
    video_score_home = video_score_home / (video_score_home + video_score_away)
    video_score_away = 1 - video_score_home
    sentiment_score_home = sentiment_score_home / (sentiment_score_home + sentiment_score_away)
    sentiment_score_away = 1 - sentiment_score_home

    # Calculate weighted scores for home and away teams
    final_score_home = (
        season_prob_home * weights['season'] +
        video_score_home * weights['video'] +
        sentiment_score_home * weights['sentiment']
    )

    final_score_away = (
        season_prob_away * weights['season'] +
        video_score_away * weights['video'] +
        sentiment_score_away * weights['sentiment']
    )

    # Determine the winner based on the final score
    if final_score_home > final_score_away:
        return f"Home team is predicted to win with a score of {final_score_home:.2f}"
    else:
        return f"Away team is predicted to win with a score of {final_score_away:.2f}"

# Example usage with your logistic regression model and scores
home_team_score = 110  # Replace with actual score data
away_team_score = 105  # Replace with actual score data
video_score_home = 0.8  # Replace with actual video analysis score
video_score_away = 0.6  # Replace with actual video analysis score
sentiment_score_home = 0.7  # Replace with actual sentiment analysis score
sentiment_score_away = 0.5  # Replace with actual sentiment analysis score

# Assuming 'model' is your trained logistic regression model
prediction = get_final_prediction(home_team_score, away_team_score, model, video_score_home, video_score_away, sentiment_score_home, sentiment_score_away)
print(prediction)


  warnings.warn(



Home team is predicted to win with a score of 0.78
